In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
import pandas as pd
from asammdf import MDF
import numpy as np
from scipy.signal import savgol_filter
import plotly.graph_objects as go
from scipy.integrate import simps

ldf is not supported
xls is not supported
xlsx is not supported


In [3]:
def parametros_seccion(df1): 
    umbral = 0.1 #con valores de aceleración mayor a 0.1 m/s2. para ia, RPA y crucero
    
    # Calcular la distancia recorrida
    df1["v_m_s_f"] = df1["v_km_h_f"]/3.6
    distance_traveled = (simps(df1["v_m_s_f"].values, df1.t_s.values))/1000  # En km
    # Calcular el tiempo total del recorrido
    total_time = (df1["t_s"].iloc[-1] - df1["t_s"].iloc[0])
    # Calcular la velocidad media
    speed_mean = (simps(df1["v_km_h_f"], df1["t_s"] * 1)) / (len(df1["t_s"]))
    # Calcular la velocidad máxima
    speed_max = df1["v_km_h_f"].max()
    # Calcular la velocidad máxima
    speed_mean_sup = (df1["v_km_h_f"].max()+speed_mean)/2
    # Calcular la aceleración media y máxima tanto negativa como positiva
    acel_pos_mean = df1[df1.acel_m_s2 > 0].acel_m_s2.mean()
    acel_pos_max = df1[df1.acel_m_s2 > 0].acel_m_s2.max()
    acel_neg_mean = df1[df1.acel_m_s2 <= 0].acel_m_s2.mean()
    acel_neg_max = df1[df1.acel_m_s2 <= 0].acel_m_s2.min()
    # Calcular tiempo en ralentí, tiempo en movimiento, tiempo en crucero y tiempo en aceleracion
    time_ralenti = (df1[(df1['v_km_h_f'] == 0)].shape[0]) * (100 / total_time)
    time_mov = (df1[(df1['v_km_h_f'] > 0)].shape[0]) * (100 / total_time)
    t_crucero = (df1[(df1['ABS_Aceleracion'] <= 0.1) & (df1['v_km_h_f'] > 0)].shape[0]) * (100 / total_time)
    t_acel = time_mov - t_crucero
    # Funcion que cuenta el numero de veces con valores de aceleración mayor a 0.1 m/s2.   
  
    df_filtered = df1[df1['acel_m_s2'] > umbral]
    n_datos_pos = len(df_filtered)
    df_filtered = df1[df1['acel_m_s2'] < -umbral]
    n_datos_neg = len(df_filtered)
    n_ia_t = n_datos_pos + n_datos_neg
    ## CALCULAR RPA
    # Calcular el producto de la velocidad por la aceleración [m3/s3 o W/kg]
    df1['vxa'] = df1['v_m_s_f']*df1["acel_m_s2"]
    # Calcular la distnacia recorrida del intervalo en metros
    distance_traveled =  (simps(df1["v_km_h_f"].divide(3.6).values,df1.t_s.values)) 
    # Calculo de velocidad media en cada intervalo         
    m_v_urban = df1['v_km_h_f'].mean()  
    # Extraer solo el dataframe con acel mayor a 0.1
    df_sort_Urban = df1[df1["acel_m_s2"]>0.1]
    # Ordenar de manera creciente de la vxa
    df_sort_vxa_urb = df_sort_Urban.sort_values(by='vxa')
    #Percentil para cada intervalo
    df_sort_vxa_urb['Percentil'] = range(1, len(df_sort_vxa_urb) + 1)
    df_sort_vxa_urb['Percentil']  = df_sort_vxa_urb['Percentil']/len(df_sort_vxa_urb)
    # interpolacion lineal para calcular el valor vxa para 0.95 en cada intervalo
    vxa_pos_urbano = df_sort_vxa_urb['vxa'].quantile(0.95, interpolation='linear')
    #Aceleració positiva relativa RPA
    RPA = (sum(df_sort_vxa_urb["vxa"]))/(distance_traveled)
    # Acumulado del valor absoluto de la derivada de la aceleracion
    ABS_Der_Acel = df1["Der_Acel_cumul"].max() #Acumulado del valor absoluto de la derivada de la aceleracion
    # Acumulado del valor absoluto de la derivada de la aceleracion por cada km recorrido
    ABS_Der_Acel_Per_distance = ABS_Der_Acel/(distance_traveled) #ABS_Der_Acel en cada km recorrido
    # Acumulado del valor absoluto de la aceleracion 
    ABS_Acel = df1['Acel_cumul'].max() #Acumulado del valor absoluto de la aceleracion
    # Acumulado del valor absoluto de la aceleracion por cada km recorrido
    ABS_Acel_Per_distance = ABS_Acel/(distance_traveled) #ABS_Acel en cada km recorrido
    
    
    #Indice de dinamismo 
    i = (ABS_Acel / n_ia_t)*(total_time*(t_acel/100))
    
    
    # Crear el DataFrame con los resultados
    results_df = pd.DataFrame(columns=["Variable", "Valor"])
    results_df["Variable"] = ["distance_traveled", "total_time", "speed_mean", "speed_max", "speed_mean_sup",
                              "acel_pos_mean", "acel_pos_max", "n_ia_pos", "n_ia_neg", "n_ia_t", "RPA", "acel_neg_mean", "acel_neg_max",
                              "time_ralenti", "time_mov","t_crucero", "t_acel", "ABS_Der_Acel", "ABS_Der_Acel/distance", "ABS_Acel", 
                              "ABS_Acel/distance", "i"]
    results_df["Valor"] = [distance_traveled, total_time, speed_mean, speed_max, speed_mean_sup,
                            acel_pos_mean, acel_pos_max, n_datos_pos, n_datos_neg, n_ia_t, RPA, acel_neg_mean, acel_neg_max,
                            time_ralenti, time_mov,t_crucero,t_acel, ABS_Der_Acel, ABS_Der_Acel_Per_distance, ABS_Acel, 
                           ABS_Acel_Per_distance, i]
    # Redondear los valores a dos decimales
    results_df["Valor"] = results_df["Valor"].round(5)
    
    return results_df


# Funcion para Calcular tiempo con velocidad por debajo de un umbral (intervalo Rural)
def umbral_ad_rural (df1):
    umbral = 60 #Velocidad minima del intervalo 
    # Calcular el tiempo total del recorrido
    total_time_1 = df1["t_s"].iloc[-1] - df1["t_s"].iloc[0]
    time_low = ((df1[df1.v_km_h_f<= umbral].shape[0]))*(100/total_time_1)
    time_high = ((df1[df1.v_km_h_f> umbral].shape[0]))*(100/total_time_1)
    print("El tiempo con velocidad por menor o igual a 60 km/h durante Intervalo Rural {:.1f} %".format(time_low))
    print("El tiempo con velocidad por mayor a 60 km/h durante Intervalo Rural {:.1f} %".format(time_high))

# Funcion para Calcular tiempo con velocidad por debajo de un umbral (intervalo Motorway)
def umbral_ad_aut (df1):
    umbral = 90 #Velocidad minima del intervalo 
    # Calcular el tiempo total del recorrido
    total_time_1 = df1["t_s"].iloc[-1] - df1["t_s"].iloc[0]
    time_low = ((df1[df1.v_km_h_f<= umbral].shape[0]))*(100/total_time_1)
    time_high = ((df1[df1.v_km_h_f> umbral].shape[0]))*(100/total_time_1)
    print("El tiempo con velocidad por menor o igual a 90 km/h durante Intervalo Motorway {:.1f} %".format(time_low))
    print("El tiempo con velocidad por mayor a 90 km/h durante Intervalo Motorway {:.1f} %".format(time_high))
    

def Calcular_distancia (df):    
    distance_m = np.zeros(len(df))
    # Calcula el cambio en el tiempo
    df['delta_tiempo'] = df['t_s'].diff().fillna(0)
    # Calcula la distancia recorrida en cada intervalo de tiempo
    df['distance_m'] = df["v_m_s_f"] * df['delta_tiempo']
    # Calcula la distancia acumulada en km
    distance_m = (df['distance_m'].cumsum())/1000 # en km   
    return (distance_m)

In [4]:
# # Abrir DFs con los datos de los intervalos

Origen = "C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/"

Equivalencias = pd.read_excel(Origen + "Parametros Intervalos/Equivalencias_Intervalos.xlsx")
Equivalencias

,NombreOriginal,NombreFinal
0,P_Interval_Motorway_05042024,Motorway_1
1,P_Interval_Motorway_15042024,Motorway_2
2,P_Interval_Motorway_29042024,Motorway_3
3,P_Interval_Rural_05042024,Rural_1
4,P_Interval_Rural_15042024,Rural_2
5,P_Interval_Rural_29042024,Rural_3
6,P_Interval_Urban_02042024,Urban_1
7,P_Interval_Urban_15042024,Urban_2
8,P_Interval_Urban_19042024,Urban_3


In [5]:
# # Abrir DFs con los datos de los intervalos

Origen = "C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/"

Equivalencias = pd.read_excel(Origen + "Parametros Intervalos/Equivalencias_Intervalos.xlsx")

Urban_1 = pd.read_excel(Origen + "Interval_Urban_02042024.xlsx")
Urban_2 = pd.read_excel(Origen + "Interval_Urban_15042024.xlsx")
Urban_3 = pd.read_excel(Origen + "Interval_Urban_19042024.xlsx")

Rural_1 = pd.read_excel(Origen + "Interval_Rural_05042024.xlsx")
Rural_2 = pd.read_excel(Origen + "Interval_Rural_15042024.xlsx")
Rural_3 = pd.read_excel(Origen + "Interval_Rural_29042024.xlsx")

Motorway_1 = pd.read_excel(Origen + "Interval_Motorway_05042024.xlsx")
Motorway_2 = pd.read_excel(Origen + "Interval_Motorway_15042024.xlsx")
Motorway_3 = pd.read_excel(Origen + "Interval_Motorway_29042024.xlsx")

In [6]:
### CICLO RDE H

# Concatenar los DFs para generar un DF con todos los datos del RDE

RDE_H = pd.concat([Urban_2, Urban_2, Rural_1, Motorway_3], axis=0)
RDE_H = RDE_H.reset_index(drop=True) # Reindexar el DF
RDE_H["t_s"] = range(len(RDE_H)) # Crear nueva columna de tiempo con una frecuencia de 1hz
# Cambiar el nombre de columna 
RDE_H.rename(columns={"distance": "distance_int"}, inplace=True)


# Concatenar los DFs para generar un DF con todos los datos del RDE

RDE_M = pd.concat([Urban_2, Urban_2, Rural_1, Motorway_2], axis=0)
RDE_M = RDE_M.reset_index(drop=True) # Reindexar el DF
RDE_M["t_s"] = range(len(RDE_M)) # Crear nueva columna de tiempo con una frecuencia de 1hz
# Cambiar el nombre de columna 
RDE_M.rename(columns={"distance": "distance_int"}, inplace=True)


### CICLO RDE L

# Concatenar los DFs para generar un DF con todos los datos del RDE

RDE_L = pd.concat([Urban_1, Urban_1, Rural_2, Motorway_1], axis=0)
RDE_L = RDE_L.reset_index(drop=True) # Reindexar el DF
RDE_L["t_s"] = range(len(RDE_L)) # Crear nueva columna de tiempo con una frecuencia de 1hz
# Cambiar el nombre de columna 
RDE_L.rename(columns={"distance": "distance_int"}, inplace=True)



In [7]:
RDE_L["distance_m"] = Calcular_distancia(RDE_L)
RDE_M["distance_m"] = Calcular_distancia(RDE_M)
RDE_H["distance_m"] = Calcular_distancia(RDE_H)

In [8]:
# Guardar parametros intervalos
RDE_L.to_excel('RDE_L_2024.xlsx', index=False)
RDE_M.to_excel('RDE_M_2024.xlsx', index=False)
RDE_H.to_excel('RDE_H_2024.xlsx', index=False)

# Abrir RDEs Guardados 

In [9]:
RDE_L = pd.read_excel('RDE_L_2024.xlsx')
RDE_M = pd.read_excel('RDE_M_2024.xlsx')
RDE_H = pd.read_excel('RDE_H_2024.xlsx')

In [29]:
# Velocidad
fig = go.Figure(data=[
    go.Scatter(x=Urban_1["t_s"], y=Urban_1["v_km_h_f"], name="Urban type 1", mode="lines", line=dict(color='darkorange')),
    go.Scatter(x=Urban_2["t_s"], y=Urban_2["v_km_h_f"], name="Urban type 2", mode="lines", line=dict(color='darkred')),
    go.Scatter(x=Urban_3["t_s"], y=Urban_3["v_km_h_f"], name="Urban type 3", mode="lines", line=dict(color='darkblue'))])

# Agregar una línea horizontal en y=60 km/h
fig.add_shape(type="line", x0=Urban_3["t_s"].iloc[0], y0=60, x1=Urban_3["t_s"].iloc[2500], y1=60,
              line=dict(color="red", width=0.5, dash="dash"),name="Umbral de velocidad")

# Agregar texto a la línea horizontal
fig.add_annotation(text="Speed <= 60 km/h ", x=Urban_3["t_s"].iloc[2300], y=64, showarrow=False, font=dict(color="red"))

# Actualizar ejes
fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Time [s]")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Speed [km/h]")

# Actualizar leyenda
fig.update_layout(legend=dict(orientation="h", x=0.3, y=1.3), height=300, width=900, plot_bgcolor="white")
fig.show()

In [30]:
# Velocidad
fig = go.Figure(data=[
    go.Scatter(x=Rural_1["t_s"], y=Rural_1["v_km_h_f"], name="Rural type 1", mode="lines", line=dict(color='darkred')),
    go.Scatter(x=Rural_2["t_s"], y=Rural_2["v_km_h_f"], name="Rural type 2", mode="lines", line=dict(color='darkorange')),
    go.Scatter(x=Rural_3["t_s"], y=Rural_3["v_km_h_f"], name="Rural type 3", mode="lines", line=dict(color='darkblue'))])

# Agregar una línea horizontal en y=60 km/h
fig.add_shape(type="line", x0=Rural_1["t_s"].iloc[0], y0=90, x1=Rural_1["t_s"].iloc[-1], y1=90,
              line=dict(color="red", width=0.5, dash="dash"),name="Umbral de velocidad")

# Agregar texto a la línea horizontal
fig.add_annotation(text="Speed <= 90 km/h ", x=Rural_2["t_s"].iloc[-1], y=96, showarrow=False, font=dict(color="red"))

# Actualizar ejes
fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Time [s]")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Speed [km/h]")

# Actualizar leyenda
fig.update_layout(legend=dict(orientation="h", x=0.3, y=1.3), height=300, width=900, plot_bgcolor="white")
fig.show()


In [31]:
# Velocidad
fig = go.Figure(data=[
    go.Scatter(x=Motorway_1["t_s"], y=Motorway_1["v_km_h_f"], name="Motorway type 1", mode="lines", line=dict(color='darkblue')),
    go.Scatter(x=Motorway_2["t_s"], y=Motorway_2["v_km_h_f"], name="Motorway type 2", mode="lines", line=dict(color='darkorange')),
    go.Scatter(x=Motorway_3["t_s"], y=Motorway_3["v_km_h_f"], name="Motorway type 3", mode="lines", line=dict(color='darkred'))])

# Agregar una línea horizontal en y=60 km/h
fig.add_shape(type="line", x0=Motorway_1["t_s"].iloc[0], y0=145, x1=Motorway_1["t_s"].iloc[-1], y1=145,
              line=dict(color="red", width=0.5, dash="dash"),name="Umbral de velocidad")

# Agregar texto a la línea horizontal
fig.add_annotation(text="Speed <= 145 km/h ", x=Motorway_2["t_s"].iloc[-90], y=157, showarrow=False, font=dict(color="red"))

# Actualizar ejes
fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Time [s]")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Speed [km/h]")

# Actualizar leyenda
fig.update_layout(legend=dict(orientation="h", x=0.3, y=1.3), height=300, width=900, plot_bgcolor="white")
fig.show()

In [13]:
# Ruta del directorio donde se encuentran los archivos Excel
directorio = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/04.End_Cycles'
directorio_guardar = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/04.End_Cycles/Parametros Ciclos'

# Lista para almacenar los nombres de los archivos Excel
archivos_excel = []

# Lista para almacenar los DataFrames con los resultados
resultados = []

# Recorrer el directorio y guardar los nombres de los archivos Excel
for archivo in os.listdir(directorio):
    if archivo.endswith('2024.xlsx'):
        archivos_excel.append(archivo)

# Iterar sobre cada archivo Excel
for archivo in archivos_excel:
    # Ruta completa del archivo
    ruta_archivo = os.path.join(directorio, archivo)
    
    # Cargar el archivo Excel en un DataFrame
    df = pd.read_excel(ruta_archivo)
    
    # Aplicar la función para calcular los parámetros de sección
    df_resultados = parametros_seccion(df)
 
    # Generar la ruta de destino para guardar el archivo modificado
    ruta_destino = os.path.join(directorio_guardar, "P_"+archivo )
    
    # Guardar el DataFrame modificado como un nuevo archivo Excel
    df_resultados.to_excel(ruta_destino, index=False)

In [14]:
import pandas as pd
import os

def obtener_nombres_archivos(directorio, extension):
    return [archivo for archivo in os.listdir(directorio) if archivo.endswith(extension)]

def obtener_nombres_completos(nombres_archivos):
    return [os.path.splitext(archivo)[0] for archivo in nombres_archivos]

def obtener_nombres_finales(nombres_archivos):
    return [nombre[0:-4]]

# Directorio donde se encuentran los archivos Excel
directorio = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/04.End_Cycles/Parametros Ciclos'

# Obtener los nombres de los archivos Excel
archivos_excel = obtener_nombres_archivos(directorio, '2024.xlsx')

# Obtener los nombres base de los archivos (sin extensión)
nombres_archivos_base = obtener_nombres_completos(archivos_excel)

# # Obtener los nombres finales de los archivos
# Nombres_r = obtener_nombres_finales(nombres_archivos_base)

# Lista para almacenar los DataFrames
dataframes = []

# Cargar los archivos Excel y cambiar el nombre de la segunda columna
for nombre_archivo in archivos_excel:
    # Cargar el archivo Excel
    ruta_archivo = os.path.join(directorio, nombre_archivo)
    df = pd.read_excel(ruta_archivo)
    # Cambiar el nombre de la segunda columna
    df.columns = [df.columns[0], nombres_archivos_base.pop(0)]
    # Agregar el DataFrame a la lista
    dataframes.append(df)

# Combinar los DataFrames en uno solo
df_final = pd.concat([dataframes[0].iloc[:, :2]] + [df.iloc[:, 1] for df in dataframes[1:]], axis=1)
# Cambiar el nombre de la columna 'A' a 'NuevoNombre'
df_final = df_final.rename(columns={'Variable': 'Parameter'})
# Mostrar el DataFrame final

print(df_final)

# guardar datos en un archivo :
df_final.to_excel('C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/04.End_Cycles/Parametros Ciclos/Parametros_Ciclos.xlsx', index=False)

                Parameter  P_RDE_H_2024  P_RDE_L_2024  P_RDE_M_2024
0       distance_traveled   99263.49699   99151.75185   99196.02963
1              total_time    6407.00000    6558.00000    6364.00000
2              speed_mean      55.76601      54.42084      56.10459
3               speed_max     135.76000     137.17000     132.17000
4          speed_mean_sup      95.76300      95.79542      94.13729
5           acel_pos_mean       0.42277       0.37778       0.38434
6            acel_pos_max       2.53472       2.32153       2.59074
7                n_ia_pos    2333.00000    2129.00000    2136.00000
8                n_ia_neg    2367.00000    2147.00000    2137.00000
9                  n_ia_t    4700.00000    4276.00000    4273.00000
10                    RPA       0.16127       0.11545       0.12048
11          acel_neg_mean      -0.37218      -0.30725      -0.33733
12           acel_neg_max      -3.12014      -3.32778      -3.12014
13           time_ralenti       7.80396       9.

In [32]:
fig = go.Figure()
# Agregar alguna traza de datos
fig.add_trace(go.Scatter(y=RDE_H["v_km_h_f"],name='Speed',mode="lines",line=dict(color='black',width=1.2)))
# Agregar la segunda traza de datos
fig.add_trace(go.Scatter(x=RDE_H["t_s"], y=RDE_H["distance_m"], name='Distance',yaxis='y2',line=dict(color='Gray',width=1.2)))

xmax = len(RDE_H["t_s"])

# Agregar una línea horizontal vel
fig.add_shape(type="line",x0=0,y0=60, x1=xmax,y1=60,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=90, x1=xmax,y1=90,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=145, x1=xmax,y1=145,line=dict(color="Red",width=0.6,dash="dashdot"),)
fig.add_annotation(x=3000, y=145, text="Speed thresholds", showarrow=True, arrowhead=1)

# Actualizar el layout para incluir el segundo eje Y
fig.update_layout(legend=dict(yanchor="top",y=1.1,xanchor="left",x=0.01),plot_bgcolor='white',paper_bgcolor='white', autosize=False,
    width=900,height=400, title_text="Accumulated speed and distance profile - RDE H",
    xaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title="Time [s]",title_standoff=10),
    yaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Speed [km/h]',title_standoff=10),
    yaxis2=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Distance [km]',title_standoff=10,
                overlaying='y',side='right',showline=True,linecolor='black',linewidth=1))
fig.show()
distance_H = (simps(RDE_H["v_km_h_f"].divide(3.6).values,RDE_H.t_s.values))/1000 # TOTAL En km
print("Distancia recorrida del ciclo RDE H {:.0f} km".format(distance_H))
total_time_H = (RDE_H["t_s"].iloc[-1] - RDE_H["t_s"].iloc[0])/60 # en minutos
print("Tiempo total del ciclo RDE H {:.1f} minutos".format(total_time_H))


fig = go.Figure()
# Agregar alguna traza de datos
fig.add_trace(go.Scatter(y=RDE_M["v_km_h_f"],name='Speed',mode="lines",line=dict(color='black',width=1.2)))
# Agregar la segunda traza de datos
fig.add_trace(go.Scatter(x=RDE_M["t_s"], y=RDE_M["distance_m"], name='Distance',yaxis='y2',line=dict(color='Gray',width=1.2)))

xmax = len(RDE_M["t_s"])

# Agregar una línea horizontal vel
fig.add_shape(type="line",x0=0,y0=60, x1=xmax,y1=60,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=90, x1=xmax,y1=90,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=145, x1=xmax,y1=145,line=dict(color="Red",width=0.6,dash="dashdot"),)
fig.add_annotation(x=3000, y=145, text="Speed thresholds", showarrow=True, arrowhead=1)

# Actualizar el layout para incluir el segundo eje Y
fig.update_layout(legend=dict(yanchor="top",y=1.1,xanchor="left",x=0.01),plot_bgcolor='white',paper_bgcolor='white', autosize=False,
    width=900,height=400, title_text="Accumulated speed and distance profile - RDE M",
    xaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title="Time [s]",title_standoff=10),
    yaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Speed [km/h]',title_standoff=10),
    yaxis2=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Distance [km]',title_standoff=10,
                overlaying='y',side='right',showline=True,linecolor='black',linewidth=1))
fig.show()

distance_L = (simps(RDE_M["v_km_h_f"].divide(3.6).values,RDE_M.t_s.values))/1000 # TOTAL En km
print("Distancia recorrida del ciclo RDE M {:.0f} km".format(distance_L))
total_time_L = (RDE_M["t_s"].iloc[-1] - RDE_M["t_s"].iloc[0])/60 # en minutos
print("Tiempo total del ciclo RDE M {:.1f} minutos".format(total_time_L))



fig = go.Figure()
# Agregar alguna traza de datos
fig.add_trace(go.Scatter(y=RDE_L["v_km_h_f"],name='Speed',mode="lines",line=dict(color='black',width=1.2)))
# Agregar la segunda traza de datos
fig.add_trace(go.Scatter(x=RDE_L["t_s"], y=RDE_L["distance_m"], name='Distance',yaxis='y2',line=dict(color='Gray',width=1.2)))

xmax = len(RDE_L["t_s"])

# Agregar una línea horizontal vel
fig.add_shape(type="line",x0=0,y0=60, x1=xmax,y1=60,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=90, x1=xmax,y1=90,line=dict(color="Red",width=0.6,dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=145, x1=xmax,y1=145,line=dict(color="Red",width=0.6,dash="dashdot"),)
fig.add_annotation(x=3000, y=145, text="Speed thresholds", showarrow=True, arrowhead=1)

# Actualizar el layout para incluir el segundo eje Y
fig.update_layout(legend=dict(yanchor="top",y=1.1,xanchor="left",x=0.01),plot_bgcolor='white',paper_bgcolor='white', autosize=False,
    width=900,height=400, title_text="Accumulated speed and distance profile - RDE L",
    xaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title="Time [s]",title_standoff=10),
    yaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Speed [km/h]',title_standoff=10),
    yaxis2=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,title='Distance [km]',title_standoff=10,
                overlaying='y',side='right',showline=True,linecolor='black',linewidth=1))
fig.show()

distance_L = (simps(RDE_L["v_km_h_f"].divide(3.6).values,RDE_L.t_s.values))/1000 # TOTAL En km
print("Distancia recorrida del ciclo RDE L {:.0f} km".format(distance_L))
total_time_L = (RDE_L["t_s"].iloc[-1] - RDE_L["t_s"].iloc[0])/60 # en minutos
print("Tiempo total del ciclo RDE L {:.1f} minutos".format(total_time_L))

Distancia recorrida del ciclo RDE H 99 km
Tiempo total del ciclo RDE H 106.8 minutos


Distancia recorrida del ciclo RDE M 99 km
Tiempo total del ciclo RDE M 106.1 minutos


Distancia recorrida del ciclo RDE L 99 km
Tiempo total del ciclo RDE L 109.3 minutos
